# MSME Forecasting - Full Notebook
This notebook is part of a 4-notebook package: Linear Regression, SARIMA, Random Forest, and Model Comparison.
Set `DATASET_INDEX` to choose which CSV from `final_msme_datasets` to use. Forecast horizon is monthly and default is 6 months.


In [1]:
# Shared imports and helper functions
import os, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print('helpers imported')
def find_csv_files(base='final_msme_datasets'):
    # return sorted csv files, handle nested folder
    if not os.path.isdir(base):
        return []
    files = glob.glob(os.path.join(base, '*.csv'))
    if len(files)==0:
        # check nested
        for d in os.listdir(base):
            p = os.path.join(base,d)
            if os.path.isdir(p):
                files += glob.glob(os.path.join(p,'*.csv'))
    return sorted(files)


helpers imported


In [2]:
# Parameters - change these if needed
DATASET_INDEX = 1  # 1-based index into final_msme_datasets folder
HORIZON = 6  # months ahead to forecast
files = find_csv_files()
print('Found', len(files), 'files')
if len(files)==0:
    raise FileNotFoundError('No CSV files found in final_msme_datasets')
dataset_path = files[DATASET_INDEX-1]
print('Using', dataset_path)
df = pd.read_csv(dataset_path)
display(df.head())


Found 5 files
Using final_msme_datasets\final_msme_dataset_1.csv


,Month-Year,Sales Revenue (₹),Service Fees (₹),Rent (₹),Utilities (₹),Salaries & Wages (₹),Raw Materials / Inventory (₹),Transportation / Logistics (₹),Loan Repayments & Interest (₹),Number of Orders,Customers,Average Order Value,Seasonality_Flag,Fuel Price Index,Total Income (₹),Total Expenses (₹),Net Cash Flow (₹)
0,2010-01,115841,44925,35802,11484,172267,167314,28048,31724,3247,3714,47.96,0,147.09,160766,446639,-285873
1,2010-02,442813,12011,30418,5675,90967,112206,16264,39003,1429,2184,253.56,0,112.71,454824,294533,160291
2,2010-03,238107,19990,28847,14722,133778,130140,27715,29995,3513,2427,60.37,1,102.72,258097,365197,-107100
3,2010-04,448831,33591,32135,5686,99071,81610,9897,8385,950,664,482.06,1,122.83,482422,236784,245638
4,2010-05,490683,42476,24191,8616,107790,109806,15783,34152,2247,916,211.96,0,80.60,533159,300338,232821


In [3]:
# Preprocessing: ensure expected columns and compute totals
expected = ['Sales Revenue (₹)','Service Fees (₹)','Rent (₹)','Utilities (₹)','Salaries & Wages (₹)','Raw Materials / Inventory (₹)','Transportation / Logistics (₹)','Loan Repayments & Interest (₹)','Number of Orders','Customers','Average Order Value','Seasonality_Flag','Fuel Price Index']
for c in expected:
    if c not in df.columns:
        df[c]=0
df['Total Income (₹)'] = df['Sales Revenue (₹)'] + df['Service Fees (₹)']
df['Total Expenses (₹)'] = df[['Rent (₹)','Utilities (₹)','Salaries & Wages (₹)','Raw Materials / Inventory (₹)','Transportation / Logistics (₹)','Loan Repayments & Interest (₹)']].sum(axis=1)
df['Net Cash Flow (₹)'] = df['Total Income (₹)'] - df['Total Expenses (₹)']
df['Month_Index'] = np.arange(1,len(df)+1)
display(df[['Month-Year','Total Income (₹)','Total Expenses (₹)','Net Cash Flow (₹)','Seasonality_Flag']].head())


,Month-Year,Total Income (₹),Total Expenses (₹),Net Cash Flow (₹),Seasonality_Flag
0,2010-01,160766,446639,-285873,0
1,2010-02,454824,294533,160291,0
2,2010-03,258097,365197,-107100,1
3,2010-04,482422,236784,245638,1
4,2010-05,533159,300338,232821,0


In [1]:
# Split into train/test (time series split)
train_size = int(0.8 * len(df))
X = df[['Month_Index']]
y_inc = df['Total Income (₹)']
y_exp = df['Total Expenses (₹)']
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_inc_train, y_inc_test = y_inc.iloc[:train_size], y_inc.iloc[train_size:]
y_exp_train, y_exp_test = y_exp.iloc[:train_size], y_exp.iloc[train_size:]
print('Train rows:', len(X_train), 'Test rows:', len(X_test))


NameError: name 'df' is not defined

In [12]:
# Train Linear Regression models
from sklearn.linear_model import LinearRegression
lr_inc = LinearRegression().fit(X_train, y_inc_train)
lr_exp = LinearRegression().fit(X_train, y_exp_train)
y_inc_pred = lr_inc.predict(X_test)
y_exp_pred = lr_exp.predict(X_test)
def metrics(y_true,y_pred):
    mae = mean_absolute_error(y_true,y_pred)
    rmse = np.sqrt(mean_squared_error(y_true,y_pred))
    r2 = r2_score(y_true,y_pred)
    return mae, rmse, r2
inc_mae, inc_rmse, inc_r2 = metrics(y_inc_test, y_inc_pred)
exp_mae, exp_rmse, exp_r2 = metrics(y_exp_test, y_exp_pred)
print('Income -> MAE:', round(inc_mae,2), 'RMSE:', round(inc_rmse,2), 'R2:', round(inc_r2,3))
print('Expenses-> MAE:', round(exp_mae,2), 'RMSE:', round(exp_rmse,2), 'R2:', round(exp_r2,3))


Income -> MAE: 100179.75 RMSE: 116759.55 R2: -0.0
Expenses-> MAE: 57153.71 RMSE: 68835.75 R2: -0.002


In [13]:
# Forecast future HORIZON months using linear trend
last_index = int(df['Month_Index'].iloc[-1])
future_idx = np.arange(last_index+1, last_index+1+HORIZON).reshape(-1,1)
future_inc = lr_inc.predict(future_idx)
future_exp = lr_exp.predict(future_idx)
future_cf = future_inc - future_exp
forecast_df = pd.DataFrame({'Future_Month_Index': future_idx.flatten(), 'Predicted_Income': future_inc.round(2), 'Predicted_Expenses': future_exp.round(2), 'Predicted_CashFlow': future_cf.round(2)})
forecast_df['Alert'] = np.where(forecast_df['Predicted_Expenses']>forecast_df['Predicted_Income'], 'Yes','No')
display(forecast_df)
os.makedirs('lr_outputs', exist_ok=True)
forecast_df.to_csv('lr_outputs/forecast_lr.csv', index=False)
pd.DataFrame([['Linear Regression','Income',inc_mae,inc_rmse,inc_r2], ['Linear Regression','Expenses',exp_mae,exp_rmse,exp_r2]], columns=['Model','Target','MAE','RMSE','R2']).to_csv('lr_outputs/metrics_lr.csv', index=False)
print('Saved LR outputs to lr_outputs/')


C:\Users\prave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\prave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,Future_Month_Index,Predicted_Income,Predicted_Expenses,Predicted_CashFlow,Alert
0,1001,329318.14,375116.07,-45797.93,Yes
1,1002,329316.20,375116.14,-45799.94,Yes
2,1003,329314.26,375116.22,-45801.95,Yes
3,1004,329312.32,375116.29,-45803.97,Yes
4,1005,329310.38,375116.36,-45805.98,Yes
5,1006,329308.44,375116.43,-45807.99,Yes


Saved LR outputs to lr_outputs/


In [14]:
# Simple rule-based recommendations
avg_aov = df['Average Order Value'].mean()
avg_orders = df['Number of Orders'].mean()
recs = []
if avg_aov < 200:
    recs.append('Consider increasing AOV via bundling/upselling or premium options')
if avg_orders < 1500:
    recs.append('Run customer acquisition & retention campaigns to increase order volume')
if df['Seasonality_Flag'].sum() > 0:
    recs.append('Plan inventory and promotions around seasonal months (Mar, Apr, Aug, Sep, Oct, Dec)')
print('Recommendations:')
for r in recs:
    print('-', r)


Recommendations:
- Consider increasing AOV via bundling/upselling or premium options
- Plan inventory and promotions around seasonal months (Mar, Apr, Aug, Sep, Oct, Dec)
